# Exploration Données de Kronyx67

### Importation

In [4]:
import pandas as pd
import plotly.express as px

### Téléchargement des données 

In [5]:
df = pd.read_csv("../data/Kronyx67.csv")

# 2. Aperçu rapide
df.head()        # premières lignes

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1759297343,"01 Oct 2025, 05:42",HUNTR/X,NaN,KPop Demon Hunters (Soundtrack from the Netfli...,17395f62-3bc4-4afd-9584-15506d51fdd1,Golden,f40cee59-ae8b-45cf-98e8-9c7278dbd821
1,1759297204,"01 Oct 2025, 05:40",Derik Fein,NaN,Hollywood,NaN,Hollywood,NaN
2,1759297023,"01 Oct 2025, 05:37",Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,Singular Act II,26d0c26d-1f74-45fd-a139-f90323d851aa,Looking at Me,bf20e876-1b34-42df-9db7-81054f4aac1a
3,1759296861,"01 Oct 2025, 05:34",Grace Power,NaN,Siren Song,NaN,Siren Song,NaN
4,1759263010,"30 Sep 2025, 20:10",Nessa Barrett,da0c8f4f-f4f4-4e91-86ca-e31108bb5706,dying on the inside,NaN,dying on the inside,eea59ae1-ad45-47f3-b5b1-145523c7b28f


In [6]:
# Convertir la colonne de temps
df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")

# Ajouter des colonnes utiles
df["date"] = df["utc_time"].dt.date
df["hour"] = df["utc_time"].dt.hour
df["weekday"] = df["utc_time"].dt.day_name()

In [8]:
df.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,date,hour,weekday
0,1759297343,2025-10-01 05:42:00,HUNTR/X,NaN,KPop Demon Hunters (Soundtrack from the Netfli...,17395f62-3bc4-4afd-9584-15506d51fdd1,Golden,f40cee59-ae8b-45cf-98e8-9c7278dbd821,2025-10-01,5,Wednesday
1,1759297204,2025-10-01 05:40:00,Derik Fein,NaN,Hollywood,NaN,Hollywood,NaN,2025-10-01,5,Wednesday
2,1759297023,2025-10-01 05:37:00,Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,Singular Act II,26d0c26d-1f74-45fd-a139-f90323d851aa,Looking at Me,bf20e876-1b34-42df-9db7-81054f4aac1a,2025-10-01,5,Wednesday
3,1759296861,2025-10-01 05:34:00,Grace Power,NaN,Siren Song,NaN,Siren Song,NaN,2025-10-01,5,Wednesday
4,1759263010,2025-09-30 20:10:00,Nessa Barrett,da0c8f4f-f4f4-4e91-86ca-e31108bb5706,dying on the inside,NaN,dying on the inside,eea59ae1-ad45-47f3-b5b1-145523c7b28f,2025-09-30,20,Tuesday


In [7]:
daily_counts = df.groupby("date").size().reset_index(name="plays")
px.line(daily_counts, x="date", y="plays", title="Écoutes par jour", markers=True)
